In [2]:
!pip install datasets
!pip install evaluate
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.3 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
dataset=load_dataset('opus_books','en-fr')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 127085
    })
})

In [5]:
dataset=dataset['train'].train_test_split(0.2)

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 101668
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 25417
    })
})

In [7]:
dataset['train'][1]

{'id': '34297',
 'translation': {'en': 'I spoke to him; he did not answer.',
  'fr': 'Je lui parlai, il ne me répondit pas.'}}

In [8]:
from transformers import AutoTokenizer
checkpoint='google-t5/t5-small'
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [9]:
source_lang='en'
target_lang='fr'
prefix='translate English to French:'
def preprocess_text(examples):
  inputs=[prefix+example[source_lang] for example in examples['translation']]
  targets=[example[target_lang] for example in examples['translation']]
  tokenized_inputs=tokenizer(inputs,max_length=128,truncation=True)
  with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)

  tokenized_inputs["labels"] = labels["input_ids"]
  return tokenized_inputs


In [10]:
preprocessed_dataset=dataset.map(preprocess_text,batched=True)

Map:   0%|          | 0/101668 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/25417 [00:00<?, ? examples/s]

In [11]:
preprocessed_dataset['train'][1]

{'id': '34297',
 'translation': {'en': 'I spoke to him; he did not answer.',
  'fr': 'Je lui parlai, il ne me répondit pas.'},
 'input_ids': [13959,
  1566,
  12,
  2379,
  10,
  196,
  5468,
  12,
  376,
  117,
  3,
  88,
  410,
  59,
  1525,
  5,
  1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [1022,
  759,
  260,
  521,
  23,
  6,
  3,
  173,
  3,
  29,
  15,
  140,
  11781,
  155,
  330,
  5,
  1]}

In [15]:
from transformers import DataCollatorForSeq2Seq
datacollator=DataCollatorForSeq2Seq(tokenizer=tokenizer,model=checkpoint,return_tensors='tf')

In [16]:
from transformers import TFAutoModelForSeq2SeqLM
model=TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
from datasets import DatasetDict,Dataset


# Dataset({
#         features: ['id', 'translation']

features_train = Dataset({
    'input_id': preprocessed_dataset['train']["input_ids"],
    'attention_mask': preprocessed_dataset['train']["attention_mask"],
    'labels': preprocessed_dataset['train']["labels"]
})

features_test = Dataset({
    'input_id': preprocessed_dataset['test']["input_ids"],
    'attention_mask': preprocessed_dataset['test']["attention_mask"],
    'labels': preprocessed_dataset['test']["labels"]
})

# Convert the empty dictionary to a DatasetDict object
dataset2 = DatasetDict({"train": features_train,"test":features_test})
print(dataset2)

In [ ]:
# preprocessed_dataset['train']
# dataset2["train"]["labels"]=preprocessed_dataset['train']["labels"]
# dataset2["train"]["input_ids"]=preprocessed_dataset['train']["input_ids"]
# dataset2["train"]["attention_mask"]=preprocessed_dataset['train']["attention_mask"]

In [ ]:
train_set=model.prepare_tf_dataset(
    # preprocessed_dataset['train'],
    dataset2["train"],
    batch_size=16,
    shuffle=True,
    collate_fn=datacollator
)
val_set=train_set=model.prepare_tf_dataset(
    # preprocessed_dataset['test'],
    dataset2["test"],
    batch_size=16,
    shuffle=False,
    collate_fn=datacollator
)

In [ ]:
from tensorflow import keras
model.compile(keras.optimizers.Adam(2e-5))

In [ ]:
import evaluate
metric=evaluate.load('sacrebleu')

In [ ]:
import numpy as np
def compute_metrics(p):
  preds,labels=p
  if isinstance(preds,tuple):
    preds=preds[0]
  decoded_preds=tokenizer.decode_batch(preds,skip_special_tokens=True)
  decoded_preds=[preds.strip() for preds in decoded_preds]
  labels=np.where(labels!=-100,labels,tokenizer.pad_token_id)
  decoded_labels=tokenizer.decode_batch(labels,skip_special_tokens=True)
  decoded_labels=[[label.strip()] for label in decoded_labels]
  result=metric.compute(predictions=decoded_preds,references=decoded_labels)
  result={'bleu':result['score']}
  predicted_lens=[np.count_nonzero(pred!=tokenizer.pad_token_id) for pred in preds]
  result['gen_len']=np.mean(predicted_lens)
  return result


In [ ]:
from transformers.keras_callbacks import KerasMetricCallback
callback=KerasMetricCallback(metric_fn=compute_metrics,eval_dataset=val_set,predict_with_generate=True,use_xla_generation=True,
    generate_kwargs={"max_length": 128})

In [ ]:
model.fit(train_set,epochs=3)

In [ ]:
input_text='translate English to French: My name is Dr. Jasmeet Singh. I teach Conversational AI'
input_ids=tokenizer(input_text,return_tensors='tf').input_ids
output=model.generate(input_ids)
tokenizer.decode(output[0],skip_special_tokens=True)